# RAG + Imagen para *Juego de Tronos* (Libro 1)

Este notebook usa:
- **LLM**: `Qwen/Qwen3-30B-A3B-Thinking-2507-FP8`
- **Imagen**: `stabilityai/stable-diffusion-3.5-large`

La lógica principal está en `got_rag_image_pipeline.py` para que sea mantenible y reutilizable.

In [ ]:
!pip -q install -U pandas pyarrow beautifulsoup4 lxml faiss-cpu sentence-transformers transformers accelerate diffusers safetensors


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
EPUB_PATH = '/content/drive/MyDrive/juego_de_tronos.epub'


In [ ]:
from got_rag_image_pipeline import (
    extract_chapters,
    build_chunks,
    save_dataframes,
    load_models,
    build_faiss_index,
    ask_and_draw,
)

chapters_df = extract_chapters(EPUB_PATH)
chunks_df = build_chunks(chapters_df, chunk_size=4500, overlap=750)
save_dataframes(chapters_df, chunks_df, out_dir='.')
print('Capítulos:', len(chapters_df), '| Chunks:', len(chunks_df))


In [ ]:
models = load_models(device='cuda')
index, _ = build_faiss_index(chunks_df, models.embedder)
print('FAISS listo:', index.ntotal)


In [ ]:
result = ask_and_draw(
    question='¿Cómo escapó Tyrion del Nido de Águilas?',
    chunks_df=chunks_df,
    index=index,
    models=models,
    seed=7,
)
print(result['answer'])
print('Prompt imagen:', result['prompt'])
result['image']
